In [1]:
import os
import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from peft import (
    LoraConfig,
    PeftConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

In [ ]:
"""import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())"""

True
1


In [64]:
# Falcon-7B model
model_id = "tiiuae/falcon-7b"  # ~13 GB (For 7B parameters)
# DistilGPT-2 model
model_id = "distilgpt2"  # ~500 MB
# GPT-2 (Small) model
#model_id = "gpt2"  # ~500 MB
# BERT-base model
#model_id = "bert-base-uncased"  # ~440 MB
# TinyBERT model
#model_id = "nreimers/TinyBERT-for-Extractive-QA"  # ~60 MB
# ALBERT-base model
#model_id = "albert-base-v2"  # ~80 MB
# T5-small model
#model_id = "t5-small"  # ~250 MB
# BART-small model
#model_id = "facebook/bart-small"  # ~250 MB

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model =AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

Unused kwargs: ['load_4bit_use_double_quant']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

c:\Users\kpreube\miniconda3\envs\llm\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kpreube\.cache\huggingface\hub\models--distilgpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [65]:
model = prepare_model_for_kbit_training(model)

In [2]:
"""config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)"""

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig
from bitsandbytes import BitsAndBytesConfig
import torch

# Load the model and tokenizer
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_id = "distilgpt2"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Configure LoRA with adjusted target modules
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["c_attn", "c_proj"],  # Correct modules for DistilGPT-2
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply PEFT (LoRA)
model = get_peft_model(model, config)


ImportError: cannot import name 'BitsAndBytesConfig' from 'bitsandbytes' (c:\Users\kpreube\miniconda3\envs\llm\Lib\site-packages\bitsandbytes\__init__.py)

In [49]:
def generate_prompt(data_point):
  return f"""
<Human>: {data_point["Context"]}
<AI>: {data_point["Response"]}
  """.strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

from datasets import load_dataset
dataset_name = 'Amod/mental_health_counseling_conversations'
dataset = load_dataset(dataset_name, split="train")

dataset = dataset.shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [50]:
training_args = transformers.TrainingArguments(
    auto_find_batch_size=True,
    num_train_epochs=4,
    learning_rate=2e-4,
    bf16=True,
    save_total_limit=4,
    logging_steps=10,
    output_dir="./output",
    save_strategy='epoch',
)

In [51]:
dataset = dataset[:100]

In [52]:
import pprint
print(dataset)

{'Context': ["I start counseling/therapy in a few days (I'm freaking out) but my main fear is that I'll cry and embarrass myself, is it something to worry about?", "I have three children. The oldest is nine and not my biological child. The middle child is three and the youngest is one. I have a great relationship with the oldest two children but I haven't developed the same level of attatchment to the youngest child. I rarely want her around and look forward to her nap times. Is this normal?", "he just walks in the house whenever he wants to see his kids. My fiancé tells me it makes her angry and doesn't like it, but she seems afraid to say anything to him about it for some reason. Is it okay for him to do this?", 'My dad is always, and I mean always, cussing and screaming at me for no reason at all. He makes me feel stupid. He also compares me to my other siblings in a negative way and demeans me. Is this abuse?', 'When my son was a teenager, we sent him to live with his dad because o

In [53]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()

  0%|          | 0/4 [00:00<?, ?it/s]

KeyError: 2

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from peft import get_peft_model, LoraConfig
from datasets import load_dataset
import torch

# Step 1: Load the GPT-2 Model and Tokenizer
model_name = "gpt2"  # Base GPT-2 model
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 has no padding token by default

# Load the model
model = AutoModelForCausalLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set the pad_token for the tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Load your dataset in a CSV file
dataset = load_dataset("csv", data_files="data.csv")

# Split the dataset into training and validation sets
train_data = dataset["train"].select([i for i in range(len(dataset["train"])) if i % 10 != 0])  # Use 90% of the data for training
val_data = dataset["train"].select([i for i in range(len(dataset["train"])) if i % 10 == 0])  # Use 10% of the data for validation

# Tokenize the input and target sequences
def tokenize_function(examples):
    inputs = tokenizer(examples['Bad_Practices'], return_tensors='pt', padding='max_length', max_length=512, truncation=True)
    labels = tokenizer(examples['Good_Practices'], return_tensors='pt', padding='max_length', max_length=512, truncation=True)
    return {'input_ids': inputs['input_ids'], 'labels': labels['input_ids']}

# Step 2: Configure LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["c_attn", "c_proj"],  # GPT-2-specific attention modules
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA
model = get_peft_model(model, lora_config)

# Step 3: Load and Prepare the Dataset
# Define prompt generation functions
def generate_prompt(data_point):
    return f"""
<Human>: {data_point["Context"]}
<AI>: {data_point["Response"]}
    """.strip()

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding="max_length", truncation=True, max_length=128)
    return {
        "input_ids": tokenized_full_prompt["input_ids"],
        "attention_mask": tokenized_full_prompt["attention_mask"],
        "labels": tokenized_full_prompt["input_ids"]  # Labels for causal LM training
    }

# Load and preprocess dataset
dataset_name = "Amod/mental_health_counseling_conversations"
dataset = load_dataset(dataset_name, split="train")
processed_dataset = dataset.shuffle(seed=42).map(generate_and_tokenize_prompt, batched=True)

# Step 4: Define Data Collator
# For causal language modeling, labels are the same as input_ids
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # No masked language modeling; causal LM task
)

# Step 5: Set Up Training Arguments and Trainer
training_args = TrainingArguments(
    output_dir="./gpt2-mental-health-finetuned",  # Output directory
    evaluation_strategy="steps",  # Evaluate every few steps
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,  # Limit to 2 checkpoints
    logging_steps=100,
    per_device_train_batch_size=4,  # Adjust based on GPU memory
    per_device_eval_batch_size=4,
    num_train_epochs=3,  # Number of epochs
    learning_rate=1e-4,  # Learning rate
    weight_decay=0.01,  # Weight decay
    warmup_steps=200,  # Warmup for LR scheduler
    logging_dir="./logs",  # Logging directory
    fp16=torch.cuda.is_available(),  # Use mixed precision if supported
    report_to="none"  # Disable logging to external tools (like W&B)
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=processed_dataset
)

# Step 6: Fine-Tune the Model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./gpt2-mental-health-finetuned")


Filter:   0%|          | 0/3512 [00:00<?, ? examples/s]

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

ArrowInvalid: Column 2 named input_ids expected length 1000 but got length 128